In [ ]:
!pip install -U spacy
!pip install spacy_transformers

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/CAD-Project/base_config.cfg /content/drive/MyDrive/CAD-Project/config.cfg

In [ ]:
def get_spacy_doc(file, data):
  nlp=spacy.blank('en')
  db = DocBin()
  for text,annot in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot['entities']
    ents=[]
    entity_indices=[]
    for start,end,label in annot:
      skip_entity=False
      for idx in range (start,end):
        if idx in entity_indices:
          skip_entity= True
          break
      if skip_entity == True :
        continue
      entity_indices = entity_indices +list(range(start,end))
      try :
        span = doc.char_span(start,end,label=label,alignment_mode='strict')
      except :
        continue
      if span is None :
        err_data = str([start,end]) + "    " +str(text) +"\n"
        file.write(err_data)
      else:
        ents.append(span)
    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass
  return db


In [ ]:
from sklearn.model_selection import train_test_split
train ,test=train_test_split(cv_data,test_size=0.3)

In [ ]:
file =open('/content/drive/MyDrive/CAD-Project/error.txt','w')
db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/CAD-Project/train_data.spacy')
db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/CAD-Project/test_data.spacy')
file.close()

In [ ]:
!python -m spacy train /content/drive/MyDrive/CAD-Project/config.cfg --output /content/drive/MyDrive/CAD-Project/output --paths.train /content/drive/MyDrive/CAD-Project/train_data.spacy --paths.dev /content/drive/MyDrive/CAD-Project/test_data.spacy --gpu-id 0

In [ ]:
nlp = spacy.load('/content/drive/MyDrive/CAD-Project/output/model-last')